# Calculating Lead Yield with Rule-Based Classification

A game company wants to create new level-based customer definitions (personas) by using some characteristics of its customers, create segments according to these new customer definitions and estimate how much new customers can bring to the company on average based on these segments.

Persona.csv data set contains the prices of the products sold by an international game company and some demographic information of the users who purchased these products. The data set consists of records created in each sales transaction. This means the table is not deduplicated. In other words, a user with certain demographic characteristics may have made more than one purchase.

In [62]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [63]:
# Read the persona.csv file and show general information about the data set.
df = pd.read_csv('persona.csv')
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [64]:
df.shape

(5000, 5)

In [65]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [66]:
# How many unique SOURCE are there? What are their frequencies?
print(df["SOURCE"].nunique())
df.SOURCE.value_counts()

2


SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [67]:
# How many unique PRICEs are there?
df["PRICE"].nunique()

6

In [68]:
# How many sales were made from which PRICE?
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [69]:
# How many sales were made from which country?
print(df["COUNTRY"].value_counts())
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64


COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

In [70]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="count")

,PRICE
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


In [71]:
#  How much was earned from sales in total by country?
df.groupby("COUNTRY")["PRICE"].sum()
df.groupby("COUNTRY").agg({"PRICE": "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [72]:
df.pivot_table(values="PRICE",index="COUNTRY",aggfunc="sum")

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [73]:
#  What are the sales numbers according to SOURCE types?
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [74]:
# What are the PRICE averages by country?
df.groupby(by=['COUNTRY']).agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [75]:
#  What are the PRICE averages according to SOURCEs?
df.groupby(by=['SOURCE']).agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [76]:
# What are the PRICE averages in the COUNTRY-SOURCE breakdown?
df.groupby(by=["COUNTRY", 'SOURCE']).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [77]:
# What are the average earnings in the COUNTRY, SOURCE, SEX, AGE breakdown?
df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [78]:
# TASK 3: Sort the output by PRICE.
agg_df = df.groupby(by=["COUNTRY", 'SOURCE', "SEX", "AGE"]).agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [79]:
#  Convert the names in the index into variable names.
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [80]:
# Convert the AGE variable to a categorical variable and add it to agg_df.

# Let's specify where the AGE variable will be divided:
bins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]

# Let's express what the naming will be in response to the divided points:
mylabels = ['0_18', '19_23', '24_30', '31_40', '41_' + str(agg_df["AGE"].max())]

agg_df["age_cat"] = pd.cut(agg_df["AGE"], bins, labels=mylabels)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


In [81]:
# TASK 6: Define new level based customers and add them to the data set as a variable.

agg_df['customers_level_based'] = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'age_cat']].agg(lambda x: '_'.join(x).upper(), axis=1)


agg_df["customers_level_based"] = ['_'.join(i).upper() for i in agg_df.drop(["AGE", "PRICE"], axis=1).values]



# variable names:
agg_df.columns

Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE', 'age_cat',
       'customers_level_based'],
      dtype='object')

In [82]:
# how do we access observation values?
for row in agg_df.values:
 print(row)

['bra' 'android' 'male' 46 59.0 '41_66'
 'BRA_ANDROID_MALE_41_66_BRA_ANDROID_MALE_41_66']
['usa' 'android' 'male' 36 59.0 '31_40'
 'USA_ANDROID_MALE_31_40_USA_ANDROID_MALE_31_40']
['fra' 'android' 'female' 24 59.0 '24_30'
 'FRA_ANDROID_FEMALE_24_30_FRA_ANDROID_FEMALE_24_30']
['usa' 'ios' 'male' 32 54.0 '31_40'
 'USA_IOS_MALE_31_40_USA_IOS_MALE_31_40']
['deu' 'android' 'female' 36 49.0 '31_40'
 'DEU_ANDROID_FEMALE_31_40_DEU_ANDROID_FEMALE_31_40']
['deu' 'android' 'female' 51 49.0 '41_66'
 'DEU_ANDROID_FEMALE_41_66_DEU_ANDROID_FEMALE_41_66']
['deu' 'ios' 'male' 20 49.0 '19_23'
 'DEU_IOS_MALE_19_23_DEU_IOS_MALE_19_23']
['usa' 'android' 'female' 47 49.0 '41_66'
 'USA_ANDROID_FEMALE_41_66_USA_ANDROID_FEMALE_41_66']
['fra' 'ios' 'male' 20 49.0 '19_23'
 'FRA_IOS_MALE_19_23_FRA_IOS_MALE_19_23']
['usa' 'ios' 'male' 59 46.5 '41_66'
 'USA_IOS_MALE_41_66_USA_IOS_MALE_41_66']
['deu' 'android' 'male' 15 46.5 '0_18'
 'DEU_ANDROID_MALE_0_18_DEU_ANDROID_MALE_0_18']
['bra' 'android' 'female' 30 45.66666

In [83]:
# We want to put the VALUES of the COUNTRY, SOURCE, SEX and age_cat variables side by side and combine them with an underscore.
[row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5].upper() for row in agg_df.values]

# Let's add it to the data set:
agg_df["customers_level_based"] = [row[0].upper() + "_" + row[1].upper() + "_" + row[2].upper() + "_" + row[5 ].upper() for row in agg_df.values]
agg_df.head()


,COUNTRY,SOURCE,SEX,AGE,PRICE,age_cat,customers_level_based
0,bra,android,male,46,59.0,41_66,BRA_ANDROID_MALE_41_66
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40


In [84]:

# Let's remove unnecessary variables:
agg_df = agg_df[["customers_level_based", "PRICE"]]
agg_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41_66,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


In [85]:
for i in agg_df["customers_level_based"].values:
 print(i.split("_"))

['BRA', 'ANDROID', 'MALE', '41', '66']
['USA', 'ANDROID', 'MALE', '31', '40']
['FRA', 'ANDROID', 'FEMALE', '24', '30']
['USA', 'IOS', 'MALE', '31', '40']
['DEU', 'ANDROID', 'FEMALE', '31', '40']
['DEU', 'ANDROID', 'FEMALE', '41', '66']
['DEU', 'IOS', 'MALE', '19', '23']
['USA', 'ANDROID', 'FEMALE', '41', '66']
['FRA', 'IOS', 'MALE', '19', '23']
['USA', 'IOS', 'MALE', '41', '66']
['DEU', 'ANDROID', 'MALE', '0', '18']
['BRA', 'ANDROID', 'FEMALE', '24', '30']
['USA', 'ANDROID', 'FEMALE', '41', '66']
['TUR', 'IOS', 'MALE', '24', '30']
['DEU', 'ANDROID', 'MALE', '31', '40']
['TUR', 'IOS', 'FEMALE', '0', '18']
['BRA', 'ANDROID', 'FEMALE', '41', '66']
['BRA', 'IOS', 'FEMALE', '19', '23']
['TUR', 'ANDROID', 'FEMALE', '31', '40']
['BRA', 'ANDROID', 'FEMALE', '31', '40']
['TUR', 'IOS', 'MALE', '31', '40']
['TUR', 'ANDROID', 'MALE', '0', '18']
['USA', 'IOS', 'MALE', '24', '30']
['BRA', 'IOS', 'MALE', '41', '66']
['DEU', 'ANDROID', 'MALE', '19', '23']
['USA', 'IOS', 'MALE', '24', '30']
['TUR', 'AN

In [86]:
print(agg_df["customers_level_based"].value_counts())

# For this reason, after groupby by segments, we should take the price averages and deduplicate the segments.
agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"})

#Located in customers_level_based index. Let's turn this into a variable.
agg_df = agg_df.reset_index()
agg_df.head()

# let's check. We expect each persona to have 1:
agg_df["customers_level_based"].value_counts()
agg_df.head()

customers_level_based
BRA_ANDROID_MALE_24_30      7
USA_ANDROID_MALE_41_66      7
USA_IOS_FEMALE_24_30        7
BRA_ANDROID_FEMALE_24_30    7
USA_ANDROID_MALE_24_30      7
BRA_IOS_MALE_31_40          6
USA_ANDROID_FEMALE_24_30    6
USA_IOS_MALE_24_30          6
BRA_IOS_FEMALE_24_30        6
USA_ANDROID_MALE_31_40      6
BRA_ANDROID_FEMALE_31_40    6
USA_IOS_FEMALE_41_66        5
BRA_ANDROID_FEMALE_19_23    5
USA_ANDROID_FEMALE_31_40    5
TUR_IOS_FEMALE_19_23        5
BRA_IOS_MALE_41_66          5
BRA_ANDROID_MALE_19_23      5
USA_ANDROID_MALE_19_23      5
USA_ANDROID_FEMALE_19_23    5
DEU_ANDROID_MALE_31_40      5
BRA_IOS_FEMALE_31_40        5
USA_IOS_MALE_41_66          5
USA_ANDROID_FEMALE_41_66    5
USA_IOS_MALE_31_40          5
USA_IOS_FEMALE_19_23        5
DEU_IOS_FEMALE_0_18         4
USA_IOS_FEMALE_0_18         4
TUR_ANDROID_FEMALE_41_66    4
BRA_ANDROID_MALE_41_66      4
BRA_IOS_MALE_0_18           4
BRA_IOS_MALE_19_23          4
BRA_IOS_FEMALE_0_18         4
DEU_ANDROID_FEMALE

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41_66,36.737179


In [87]:

# Segment new customers (USA_ANDROID_MALE_0_18).

agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_FEMALE_0_18,35.645303,B
1,BRA_ANDROID_FEMALE_19_23,34.077340,C
2,BRA_ANDROID_FEMALE_24_30,33.863946,C
3,BRA_ANDROID_FEMALE_31_40,34.898326,B
4,BRA_ANDROID_FEMALE_41_66,36.737179,A
5,BRA_ANDROID_MALE_0_18,34.805861,B
6,BRA_ANDROID_MALE_19_23,31.673243,D
7,BRA_ANDROID_MALE_24_30,33.413919,C
8,BRA_ANDROID_MALE_31_40,34.327381,B
9,BRA_ANDROID_MALE_41_66,40.041667,A


In [88]:
agg_df.groupby("SEGMENT").agg({"PRICE": "mean"})

,PRICE
SEGMENT,
D,29.206780
C,33.509674
B,34.999645
A,38.691234


In [89]:
# TASK 8: Classify new customers and estimate how much income they can bring.

# To which segment does a 33-year-old Turkish woman using ANDROID belong and how much income is she expected to earn on average?
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]


,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [90]:
# In which segment and how much income on average is a 35-year-old French woman using IOS expected to earn?
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C
